In [2]:
import pandas as pd
import numpy as np
import nltk
import re

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from joblib import dump, load
from sklearn.metrics import classification_report
from nltk import word_tokenize
from nltk import WordNetLemmatizer


nltk.download('vader_lexicon')
nltk.download('twitter_samples')
nltk.download('punkt')

# importieren der Trainigsdaten---nltk stellt tweets dafür bereit
from nltk.corpus import twitter_samples

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\misarasa\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\misarasa\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\misarasa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
pos_twee = twitter_samples.strings('positive_tweets.json')
neg_twee = twitter_samples.strings('negative_tweets.json') 
text = twitter_samples.strings('tweets.20150430-223406.json')

In [4]:
print(pos_twee[0]) #Ausgabe des ersten Tweets

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [5]:
print(len(text))
print(len(pos_twee))
print(len(neg_twee))

20000
5000
5000


In [6]:
df = pd.DataFrame (pos_twee,columns=['text'])
df['Value'] = 1
df2 = pd.DataFrame (neg_twee,columns=['text'])
df2['Value'] = -1

df

,text,Value
0,#FollowFriday @France_Inte @PKuchly57 @Milipol...,1
1,@Lamb2ja Hey James! How odd :/ Please call our...,1
2,@DespiteOfficial we had a listen last night :)...,1
3,@97sides CONGRATS :),1
4,yeaaaah yippppy!!! my accnt verified rqst has...,1
...,...,...
4995,"@chriswiggin3 Chris, that's great to hear :) D...",1
4996,@RachelLiskeard Thanks for the shout-out :) It...,1
4997,@side556 Hey! :) Long time no talk...,1
4998,@staybubbly69 as Matt would say. WELCOME TO AD...,1


In [7]:
df2

,text,Value
0,hopeless for tmr :(,-1
1,Everything in the kids section of IKEA is so c...,-1
2,@Hegelbon That heart sliding into the waste ba...,-1
3,"“@ketchBurning: I hate Japanese call him ""bani...",-1
4,"Dang starting next week I have ""work"" :(",-1
...,...,...
4995,I wanna change my avi but uSanele :(,-1
4996,MY PUPPY BROKE HER FOOT :(,-1
4997,where's all the jaebum baby pictures :((,-1
4998,But but Mr Ahmad Maslan cooks too :( https://t...,-1


In [8]:
df=df.append(df2, ignore_index = True)
df = df.sample(frac=1).reset_index(drop=True) # mischen der positiven und negativen tweets

In [9]:
df


,text,Value
0,Happy Friday :-) http://t.co/vEZV71ZpW5,1
1,What the hell is going wrong :( I don't unders...,-1
2,#FollowFriday @CCIdeLYON @CCI_95 @CPNERI_ESIGR...,1
3,I've been going to sleep like at 5am everday :...,-1
4,@ThreeStrypes Yay new vid!!! :D,1
...,...,...
9995,"@FafnirsWind Maybe there are Garrus, Liara and...",1
9996,"The starbucks logo stands for obsession , addi...",1
9997,@parapluieverte Glad you enjoyed your journey!...,1
9998,I seem to have burned everything apart from th...,-1


# Bereinigen der Daten

In [10]:
remove_rt = lambda x: re.sub("(RT @\w+: )|(@\w+):"," ",x) # herausfiltern der usernamen und RT
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|[\d\.]|(\w+:\/\/\S+)|(\n|\t|\')"," ",x) # herausfiltern markierter Usernamen 
htp = lambda x: re.sub('http[s]?:\/\/(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'," ",x) # entfernen von links, sonderzeichen, punkten

df["text"] = df.text.map(remove_rt).map(rt).map(htp)
df["text"] = df.text.str.lower()
df.text.head(10)

0                                   happy friday :-)  
1    what the hell is going wrong :( i don t unders...
2    #followfriday    _    _esigroup for being top ...
3    i ve been going to sleep like at  am everday :...
4                                    yay new vid!!! :d
5     _official really happy today       wat u done...
6      but but out of all the days, same day pa :( ...
7                           @ sostumblrx the apma s :(
8                                   ye to #pk hay :p  
9    spotify finally figured out that i m not in th...
Name: text, dtype: object

In [11]:
# bereinigen des textes von den Emojis
def deEmojify(text):
    regrex_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'',text)

df["text"] = df.text.map(deEmojify)

In [12]:
data = list(df["text"])
labels = list(df["Value"])
data[:5]

['happy friday :-)  ',
 'what the hell is going wrong :( i don t understand why people choose to hurt each other  ',
 '#followfriday    _    _esigroup for being top engaged members in my community this week :)',
 'i ve been going to sleep like at  am everday :( it would be cool if other people like me would hangout at this time!',
 '  yay new vid!!! :d']

In [13]:
print(len(data))
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.15, random_state=42)

10000


In [14]:
x_train[:5]

['@_pearlyyy well, kind of :) i still read it if the author is my favorite,otherwise,i tend to not read them, that s why i crave for more fics',
 'the amber liquid continues to flow, almost game time :-) #melbournetrip     #taxikitchen  ',
 '  this :) is :) so :) relevant :)))))))))) i :) want :) to :) puke :)))',
 '    it keeps better and better every update :)',
 '  _abla  _messiah because people who point out inaccuracies in jokes might be inaccurate jokes  i got worried :(']

# Erstellung und Optimierung des Modells

In [15]:
import warnings
warnings.filterwarnings('ignore')

#Diese Wörter werden ignoriert
stopwords = ["able","about","above","abroad","according","accordingly","across","actually","adj","after","afterwards","again","against","ago","ahead","ain't","all","allow","allows","almost","alone","along","alongside","already","also","although","always","am","amid","amidst","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","a's","aside","ask","asking","associated","at","available","away","awfully","back","backward","backwards","be","became","because","become","becomes","becoming","been","before","beforehand","begin","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","came","can","cannot","cant","can't","caption","cause","causes","certain","certainly","changes","clearly","c'mon","co","co.","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't","course","c's","currently","dare","daren't","definitely","described","despite","did","didn't","different","directly","do","does","doesn't","doing","done","don't","down","downwards","during","each","edu","eg","eight","eighty","either","else","elsewhere","end","ending","enough","entirely","especially","et","etc","even","ever","evermore","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","fairly","far","farther","few","fewer","fifth","first","five","followed","following","follows","for","forever","former","formerly","forth","forward","found","four","from","further","furthermore","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","had","hadn't","half","happens","hardly","has","hasn't","have","haven't","having","he","he'd","he'll","hello","help","hence","her","here","hereafter","hereby","herein","here's","hereupon","hers","herself","he's","hi","him","himself","his","hither","hopefully","how","howbeit","however","hundred","i'd","ie","if","ignored","i'll","i'm","immediate","in","inasmuch","inc","inc.","indeed","indicate","indicated","indicates","inner","inside","insofar","instead","into","inward","is","isn't","it","it'd","it'll","its","it's","itself","i've","just","k","keep","keeps","kept","know","known","knows","last","lately","later","latter","latterly","least","less","lest","let","let's","like","liked","likely","likewise","little","look","looking","looks","low","lower","ltd","made","mainly","make","makes","many","may","maybe","mayn't","me","mean","meantime","meanwhile","merely","might","mightn't","mine","minus","miss","more","moreover","most","mostly","mr","mrs","much","must","mustn't","my","myself","name","namely","nd","near","nearly","necessary","need","needn't","needs","neither","never","neverf","neverless","nevertheless","new","next","nine","ninety","no","nobody","non","none","nonetheless","noone","no-one","nor","normally","not","nothing","notwithstanding","novel","now","nowhere","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","one's","only","onto","opposite","or","other","others","otherwise","ought","oughtn't","our","ours","ourselves","out","outside","over","overall","own","particular","particularly","past","per","perhaps","placed","please","plus","possible","presumably","probably","provided","provides","que","quite","qv","rather","rd","re","really","reasonably","recent","recently","regarding","regardless","regards","relatively","respectively","right","round","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","shan't","she","she'd","she'll","she's","should","shouldn't","since","six","so","some","somebody","someday","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","take","taken","taking","tell","tends","th","than","thank","thanks","thanx","that","that'll","thats","that's","that've","the","their","theirs","them","themselves","then","thence","there","thereafter","thereby","there'd","therefore","therein","there'll","there're","theres","there's","thereupon","there've","these","they","they'd","they'll","they're","they've","thing","things","think","third","thirty","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","till","to","together","too","took","toward","towards","tried","tries","truly","try","trying","t's","twice","two","un","under","underneath","undoing","unfortunately","unless","unlike","unlikely","until","unto","up","upon","upwards","us","use","used","useful","uses","using","usually","v","value","various","versus","very","via","viz","vs","want","wants","was","wasn't","way","we","we'd","welcome","well","we'll","went","were","we're","weren't","we've","what","whatever","what'll","what's","what've","when","whence","whenever","where","whereafter","whereas","whereby","wherein","where's","whereupon","wherever","whether","which","whichever","while","whilst","whither","who","who'd","whoever","whole","who'll","whom","whomever","who's","whose","why","will","willing","wish","with","within","without","wonder","won't","would","wouldn't","yes","yet","you","you'd","you'll","your","you're","yours","yourself","yourselves","you've","zero","a","how's","i","when's","why's","b","c","d","e","f","g","h","j","l","m","n","o","p","q","r","s","t","u","uucp","w","x","y","z","I","www","amount","bill","bottom","call","computer","con","couldnt","cry","de","describe","detail","due","eleven","empty","fifteen","fifty","fill","find","fire","forty","front","full","give","hasnt","herse","himse","interest","itse”","mill","move","myse”","part","put","show","side","sincere","sixty","system","ten","thick","thin","top","twelve","twenty","abst","accordance","act","added","adopted","affected","affecting","affects","ah","announce","anymore","apparently","approximately","aren","arent","arise","auth","beginning","beginnings","begins","biol","briefly","ca","date","ed","effect","et-al","ff","fix","gave","giving","heres","hes","hid","home","id","im","immediately","importance","important","index","information","invention","itd","keys","kg","km","largely","lets","line","'ll","means","mg","million","ml","mug","na","nay","necessarily","nos","noted","obtain","obtained","omitted","ord","owing","page","pages","poorly","possibly","potentially","pp","predominantly","present","previously","primarily","promptly","proud","quickly","ran","readily","ref","refs","related","research","resulted","resulting","results","run","sec","section","shed","shes","showed","shown","showns","shows","significant","significantly","similar","similarly","slightly","somethan","specifically","state","states","stop","strongly","substantially","successfully","sufficiently","suggest","thered","thereof","therere","thereto","theyd","theyre","thou","thoughh","thousand","throug","til","tip","ts","ups","usefully","usefulness","'ve","vol","vols","wed","whats","wheres","whim","whod","whos","widely","words","world","youd","youre"]

class lemmaTokenizer: # Macht Konjugation rückgängig, allderdings nur auf Englisch
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

pipe = make_pipeline(TfidfVectorizer(), MultinomialNB())
#Vectorizer/Modell kann testweise hier geändert werden

pipe.fit(x_train, y_train) # Training ohne Hyperparameteroptimierung

print('-------------------------- Unoptimiert -------------------------\n')
print(classification_report(y_test, pipe.predict(x_test), digits=4)) 

param_grid = {'multinomialnb__alpha': [0.01, 0.1, 0.25, 0.5, 1],
'multinomialnb__fit_prior' : [True, False],
'tfidfvectorizer__use_idf' : [True, False],
'tfidfvectorizer__norm': ['l1', 'l2'],
'tfidfvectorizer__stop_words' : [stopwords, None],                      
'tfidfvectorizer__tokenizer': [lemmaTokenizer, None],              
'tfidfvectorizer__ngram_range': [(1,1), (1,2), (2,2), (2,3), (3,3)]}


gridsearch = GridSearchCV(pipe, param_grid, scoring = 'f1', cv=10, n_jobs = 7)
gridsearch.fit(x_train, y_train)
print("-------------------------- Grid Search -------------------------\n")
print(classification_report(y_test, gridsearch.predict(x_test), digits=4))


randomsearch = RandomizedSearchCV(pipe, param_grid, scoring = 'f1', cv=10, n_jobs = 7, n_iter = 100)
randomsearch.fit(x_train, y_train)
print("-------------------------- Random Search -----------------------\n")
print(classification_report(y_test, randomsearch.predict(x_test), digits=4))

#dump(randomsearch, 'model.joblib') # Abspeichern des besten Modells, um es auf langsamen Geräten nicht erneut trainieren zu müssen

-------------------------- Unoptimiert -------------------------

              precision    recall  f1-score   support

          -1     0.7620    0.7880    0.7748       764
           1     0.7718    0.7446    0.7580       736

    accuracy                         0.7667      1500
   macro avg     0.7669    0.7663    0.7664      1500
weighted avg     0.7668    0.7667    0.7665      1500

-------------------------- Grid Search -------------------------

              precision    recall  f1-score   support

          -1     0.7699    0.7840    0.7769       764
           1     0.7715    0.7568    0.7641       736

    accuracy                         0.7707      1500
   macro avg     0.7707    0.7704    0.7705      1500
weighted avg     0.7707    0.7707    0.7706      1500

-------------------------- Random Search -----------------------

              precision    recall  f1-score   support

          -1     0.7699    0.7840    0.7769       764
           1     0.7715    0.7568    0.

# Anreichern der Tweets

In [16]:
#einlesen der Daten
data_tweets = pd.read_json("./doge_twitterData.json", lines=True)
df_tweets=data_tweets


In [17]:
#bereinigen der Tweets
remove_rt = lambda x: re.sub("(RT @\w+: )|(@\w+):"," ",x) #herausfiltern der usernamen und RT
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|[\d\.]|(\w+:\/\/\S+)|(\n|\t|\')"," ",x) #herausfiltern markierter Usernamen 
htp = lambda x: re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'," ",x) #entfernen von links, sonderzeichen, punkten

df_tweets["text"] = df_tweets.text.map(remove_rt).map(rt).map(htp)
df_tweets["text"] = df_tweets.text.str.lower()
df_tweets.text.head(10)

#bereinigen des Textes von den Emojis
def deEmojify(text):
    regrex_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'',text)

df_tweets["text"] = df_tweets.text.map(deEmojify)

df_tweets.drop(df_tweets[df_tweets['lang'] != 'en'].index, inplace = True) #Modell ist nur für englisch trainiert -> nur Englisch auswerten

In [18]:
df_tweets[:4]

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,display_text_range,extended_tweet,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities
0,2021-06-28 16:25:33+00:00,1409548535272407047,1409548535272407040,$ in babydoge giveaway alert to enter r...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,2021-06-28 16:25:33.611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-06-28 16:25:33+00:00,1409548536740552712,1409548536740552704,bitcoin could collapse el salvador s econom...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,2021-06-28 16:25:33.961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-06-28 16:25:34+00:00,1409548538988609537,1409548538988609536,amen brother! #dogecoin #tothemoon,"<a href=""http://twitter.com/download/android"" ...",False,1.409546e+18,1.409546e+18,8.055323e+17,8.055323e+17,...,en,2021-06-28 16:25:34.497,"[16, 51]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-06-28 16:25:34+00:00,1409548540234194947,1409548540234194944,the most powerful doge of them all is about ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.406900e+18,1.406900e+18,1.074695e+18,1.074695e+18,...,en,2021-06-28 16:25:34.794,"[13, 140]",{'full_text': '@BitcoinAssn The most powerful ...,0.0,NaN,NaN,NaN,NaN,NaN


In [19]:
clean_data_list = list(df_tweets["text"])
clean_data_list[:5]

['  $    in babydoge giveaway alert   to enter retweet &amp; follow!  ⏱    hours only!⏱   ticeans, it’s the dailytice monday gi…',
 ' bitcoin  could collapse  el salvador s economy   #coinmarketcap #crypto #cryptocurrency #bitcoin #altcoin #defi #dogeco…',
 '     amen brother! #dogecoin  #tothemoon',
 '  the most powerful doge of them all is about to presale on tuesday! exclusive autoboost function! don t…  ',
 ' rakucoin $rakuc currently on eth and just launched on bsc   missed dogecoin and shiba inu? rakucoin your next    x potential …']

In [20]:
prob_single=randomsearch.predict_proba(list(clean_data_list)) 
#Der Array enthält die jeweiligen vorhergesagenen W. für beide Klassen
prob_single

array([[0.21289138, 0.78710862],
       [0.44427515, 0.55572485],
       [0.36404585, 0.63595415],
       ...,
       [0.26690087, 0.73309913],
       [0.41837708, 0.58162292],
       [0.06941589, 0.93058411]])

In [21]:
neg_prob=[]
pos_prob=[]
print(type(neg_prob))

for i in prob_single: #abspeichern der Wahrscheinlichkeiten in Listen
    
    neg_prob.append(i[0]) 
    pos_prob.append(i[1])



<class 'list'>


In [22]:
data_tweets["neg_prob"]=neg_prob #die erstellte Liste wird in die Entspechenden Spalte eingefügt
data_tweets["pos_prob"]=pos_prob

In [23]:
data_tweets[:5]

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,display_text_range,extended_tweet,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,neg_prob,pos_prob
0,2021-06-28 16:25:33+00:00,1409548535272407047,1409548535272407040,$ in babydoge giveaway alert to enter r...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.212891,0.787109
1,2021-06-28 16:25:33+00:00,1409548536740552712,1409548536740552704,bitcoin could collapse el salvador s econom...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.444275,0.555725
2,2021-06-28 16:25:34+00:00,1409548538988609537,1409548538988609536,amen brother! #dogecoin #tothemoon,"<a href=""http://twitter.com/download/android"" ...",False,1.409546e+18,1.409546e+18,8.055323e+17,8.055323e+17,...,"[16, 51]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.364046,0.635954
3,2021-06-28 16:25:34+00:00,1409548540234194947,1409548540234194944,the most powerful doge of them all is about ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.406900e+18,1.406900e+18,1.074695e+18,1.074695e+18,...,"[13, 140]",{'full_text': '@BitcoinAssn The most powerful ...,0.0,NaN,NaN,NaN,NaN,NaN,0.443402,0.556598
4,2021-06-28 16:25:34+00:00,1409548540494245896,1409548540494245888,rakucoin $rakuc currently on eth and just lau...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.476097,0.523903


In [24]:
data_tweets["sentiment"]= 0 #ertsellen einer neuen Spalte

#je nache Wahscheinlichketi wird der Tweet einer der 3 Klassen zugewiesen 
for index, neg in data_tweets["neg_prob"].iteritems(): 
    
    
    if neg > 0.55 :
        data_tweets.loc[index,"sentiment"]="negative"

    elif neg < 0.45:
        data_tweets.loc[index,"sentiment"]= "positive"

    else:
        data_tweets.loc[index,"sentiment"]= "neutral"
        
    
 

In [25]:
data_tweets[:5]

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,extended_tweet,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities,neg_prob,pos_prob,sentiment
0,2021-06-28 16:25:33+00:00,1409548535272407047,1409548535272407040,$ in babydoge giveaway alert to enter r...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.212891,0.787109,positive
1,2021-06-28 16:25:33+00:00,1409548536740552712,1409548536740552704,bitcoin could collapse el salvador s econom...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.444275,0.555725,positive
2,2021-06-28 16:25:34+00:00,1409548538988609537,1409548538988609536,amen brother! #dogecoin #tothemoon,"<a href=""http://twitter.com/download/android"" ...",False,1.409546e+18,1.409546e+18,8.055323e+17,8.055323e+17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.364046,0.635954,positive
3,2021-06-28 16:25:34+00:00,1409548540234194947,1409548540234194944,the most powerful doge of them all is about ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.406900e+18,1.406900e+18,1.074695e+18,1.074695e+18,...,{'full_text': '@BitcoinAssn The most powerful ...,0.0,NaN,NaN,NaN,NaN,NaN,0.443402,0.556598,positive
4,2021-06-28 16:25:34+00:00,1409548540494245896,1409548540494245888,rakucoin $rakuc currently on eth and just lau...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.476097,0.523903,neutral


In [31]:
data_tweets.to_json('./plots_visualizations/tweets_with_sentiment.json') #Speichern der bereicherten DataFrames